## **실습 4.**

본 실습 자료는 **BDC103(00) 빅데이터와정보검색 강의 실습**을 위해 **고려대학교 자연어처리연구실 (NLP & AI Lab)**에서 제작했습니다.

☠️ 외부로의 무단 배포를 금지합니다. ☠️

```
version 1.0 (2021.05.28)
created by: 서재형, 임희석 (고려대학교 자연어처리 연구실)
email: wolhalang@gmail.com
```

```
HuggingFace의 모델과 학습 방법을 참조해서 제작했습니다.
https://huggingface.co/transformers/
```

## 수업 시작 전! 😀

BDC103_IR 깃헙 저장소에서 **final_test.json**와 **실습 4강.ipynb**를 다운로드 받으시고

본인의 구글 드라이브 **Colab Notebooks/information_retrieval** 폴더로 옮겨주세요

### **구글 드라이브 연동** 😸

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive/') 

os.chdir('/content/gdrive/My Drive/Colab Notebooks/information_retrieval') ## 현재 작업 환경으로 설정한 경로를 입력하세요

Mounted at /content/gdrive/


### **필요한 패키지 불러오기** 😼


In [ ]:
# 설치하는 항목들
!pip install datasets
!pip install transformers
!pip install accelerate

# 로드하는 항목들
import argparse
import math
import os
import random
import datasets
import numpy as np
import torch
import collections
import json
import logging
import transformers
import easydict
import pandas as pd

from tqdm import tqdm_notebook
from tqdm.auto import tqdm
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets, load_metric
from torch.utils.data.dataloader import DataLoader
from typing import Optional, Tuple
from accelerate import Accelerator

# Huggingface transformers의 토크나이저, 설정파일, 최적화 함수 등 사용
from transformers import (
    CONFIG_MAPPING,
    MODEL_MAPPING,
    AdamW,
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    SchedulerType,
    default_data_collator,
    get_scheduler,
    set_seed,
)
from utils_qa import postprocess_qa_predictions
from transformers.utils import check_min_version

     |████████████████████████████████| 235kB 6.5MB/s 
     |████████████████████████████████| 112kB 8.8MB/s 
     |████████████████████████████████| 245kB 9.3MB/s 
     |████████████████████████████████| 2.3MB 6.5MB/s 
     |████████████████████████████████| 901kB 27.5MB/s 
     |████████████████████████████████| 3.3MB 34.9MB/s 
  Found existing installation: huggingface-hub 0.0.9
    Uninstalling huggingface-hub-0.0.9:
      Successfully uninstalled huggingface-hub-0.0.9
     |████████████████████████████████| 51kB 2.0MB/s 


## 📌 Step 1. 캐글 활용하기 ☝️

## 1) 캐글 입문 🐣

**Kaggle이란?**

데이터 분석 및 머신 러닝 등 데이터 사이언스를 주제로 하는 경진 대회 (Competition)을 주관하는 플랫폼.

개인 또는 기관에서 데이터 관련 해결 과제를 상금과 함께 의뢰하고, 이에 대해서 전 세계의 데이터 사이언티스트들이 함께 문제를 해결하고 경쟁합니다.

Kaggle은 단순 경쟁을 넘어서, 여러 가지 주제에 따른 다양한 데이터에 분석 방법에 있어서 자유로운 공유와 토의가 이루어지면서 입문자 ~ 전문가 모두에게 유익한 정보를 제공하는데 큰 의미를 지닙니다.

<br>

**그렇다면 우리는 무엇을 할 수 있을까?**

Kaggle에서 우리는 크게 2가지의 Task를 할 수 있습니다.
 
(1) 지난 경연, Toy 데이터, 또는 파생 데이터 분석
--> 추가적인 의뢰에 의해 누군가 상금을 새롭게 제시할 수 있지만, 기본적으로는 **연습 훈련**에 가깝습니다.

(2) 경연 참가 
--> 어느정도 실력과 기회가 닿았을 때 도전하는 것이 일반적이며, 상금 사냥, 포트 폴리오, 실력 검증, 시험 등을 목표로 하는 **실전**에 가깝습니다


<br>

**Kaggle 홈페이지**

https://www.kaggle.com/

<br>

---------

**[최초 화면]**

- Kaggle 링크를 클릭해서 들어간 이후 자신의 Google 계정으로 로그인한 상태
- 좌측의 메뉴바를 클릭하면 아래 사진과 같이 주요 메뉴가 등장합니다.

---------

**Compete**: 지난, 현재 진행 중인 경연 정보를 확인할 수 있습니다.

---------

**Datasets**: 분석 가능한 데이터셋과 그에 대한 정보를 확인할 수 있습니다


---------

**Notebooks**: 특정 데이터셋 및 주제에 대해 작성한 '다른 사람의 코드', '자신이 작성한 코드' 그리고 웹 상의 Kernel을 통해서 자신의 새로운 Notebook 상의 코드를 작성할 수 있습니다.

---------

**Discuss**: 특정 데이터셋 및 주제에 대한 Q&A와 논의점들을 확인할 수 있습니다.


---------

**Courses**: Python Tutorial, Pandas, Numpy 등 데이터 사이언스를 위해 필요한 강좌가 개설되어 있습니다. 










## 2) 필요한 Dataset을 Kaggle에서 확인해보기 🐥

**데이터 또는 주제 검색**

상단의 검색 메뉴를 통해서 특정 키워드를 입력하면 관련된 주제 또는 이름을 지닌 데이터셋 정보를 확인합니다.

ex) *titanic*

https://www.kaggle.com/c/titanic

---

**필요 데이터셋 또는 주제를 찾았다면?**

위의 링크에서 커버 사진 아래의 메뉴 버튼 (ex. Overview)을 클릭한 뒤 스크롤해서 내리다보면 필요한 정보를 얻을 수 있습니다.

---

**Data**

데이터에 대한 설명이 되어 있는 코드북과, 어떠한 목표와 특징을 가지고 제작된 것인지, 그리고 데이터셋에 대한 간단한 미리보기와 함께 다운로드 받을 수 있도록 되어 있습니다.

---

**Tasks**

Competition이 아닌 경우, 해당 데이터셋에 대해서 개인 또는 기관 단위에서 특정한 의뢰와 함께 상금을 제시합니다.

---

**Kernels**

해당 데이터셋 또는 주제에 대해 다른 사람들이 작성한 코드와 주석을 볼 수 있습니다. 처음부터 백지 상태에서 모든 것을 작성하는 것보다 선행자들이 분석한 내용을 참고하여 나의 분석 목적과 의도에 맞게 수정해나가는 것도 좋은 방법입니다.

(다른 사람의 코드를 해석해서 벤치마크하여 나에게 맞도록 수정하는 것도 절대 쉬운 일이 아닙니다.)

---

**Discussion**

어쩌면 지금 내가하고 있는 고민은 이미 다른 사람들도 똑같이 했었고 이미 해결한 문제일 수 있습니다. 직접 질문을 할 수도 있는 만큼, 자신이 하고 있는 고민이 구글링으로도 해결하기 어려운 일이라면 한번 확인해보는 것도 나쁘지 않습니다.









## 3) Competition 참가 🐤

**Active / Completed / InClass**

현재 진행 중이며, 상금이 걸려 있는 주제와 데이터셋에 무엇이 있는지 확인할 수 있습니다. 또한, 지난 경연과 대학교 강의 등 교육 목적으로 개최한 경연 대회 목록을 확인할 수 있습니다.

---

**Team**

그러나, 전반적으로 사용자가 확인하고 활용할 수 있는 정보에 대해서는 사실상 큰 차이가 없습니다. 다만 여기는 실전!! 팀을 만들어서 경연 대회에 참여할 수 있습니다. (파티 퀘스트?)

---

**Leaderboard**

Competition의 꽃!! 아래에 보이는 리더 보드는 어느 모델, 그리고 어떠한 팀이 가장 잘 했는지 지정한 기준에 따라서 순위를 매깁니다. 

데이터 사이언스... 머신 러닝.. 대부분의 발전은 이러한 경쟁에서 비롯되었다고 해도 과언이 아닙니다.

---

**Submit**

1번 방법으로 로컬 환경에서 직접 실행한 결과를 담은 정답 csv를 만든 후 업로드할 수 있습니다. 

2번 방법으로는 앞서 언급했던 Kernels 탭에서, 웹 상에서 작성하고 분석한 결과를 온라인에서 바로 제출할 수 있습니다.











## 4) Kaggle로 Final Term 해결하기 😎

**Document Retrieval Question Anwsering**

https://www.kaggle.com/c/bdc103-final

위의 링크에서 데이터셋을 다운로드 받고,

- final_test_sample.json (이번 실습에서는 해당 파일만 사용)
- final_term_no_context.json

위의 두 개 파일을 자신의 'Google Drive' 상의 'information_retrieval' 폴더에 옮겨보자.


In [ ]:
## 명령행 인터페이스, 사전 훈련된 모델을 로드하여 훈련하고 평가하는데까지 필요한 인자를 사용자 정의 인자로 설정하는 내용을 담고 있음.
## 사용법: python example.py --dataset_name squad.json .... 
## 주피터 환경을 위한 인자 설정법 
import easydict
import string
import re
def easydict_args():
    
    args = easydict.EasyDict({
        "dataset_name": None,
        "dataset_config_name": 'squad',
        "train_file": 'final_test_no_context.json',
        "preprocessing_num_workers": 4,
        "validation_file": 'final_test_no_context.json',
        "test_file": 'final_test_no_context.json',
        "test_save_path" : "result_final/final_test.csv",
        "max_seq_length": 384,
        "pad_to_max_length": None,
        "model_name_or_path": 'result/pytorch_model.bin',
        "config_name": 'bert-base-cased',
        "tokenizer_name": 'bert-base-cased',
        "use_slow_tokenizer": None,
        "per_device_train_batch_size": 4,
        "per_device_eval_batch_size": 4,
        "learning_rate": 3e-5,
        "weight_decay": 0.0,
        "num_train_epochs": 3,
        "max_train_steps": None,
        "gradient_accumulation_steps": 1,
        "lr_scheduler_type": 'linear',
        "num_warmup_steps":0,
        "output_dir": 'result_final/',
        "seed": 42,
        "doc_stride": 128,
        "n_best_size": 20,
        "null_score_diff_threshold": 0.0,
        "version_2_with_negative": False,
        "max_answer_length": 30,
        "max_train_samples": None,
        "max_eval_samples": None,
        "overwrite_cache": False,
        "max_predict_samples": None,
        "do_train": False,
        "do_eval": True,
        "do_predict": False,
    })

    ## 무결성 체크
    ## 인자로 전달한 데이터셋 명이 잘못되었거나 확장자 오류가 있을 경우 raise 이하의 메세지를 출력
    if (
        args.dataset_name is None
        and args.train_file is None
        and args.validation_file is None
        and args.test_file is None
    ):
        raise ValueError("Need either a dataset name or a training/validation/test file.")
    else:
        if args.train_file is not None:
            extension = args.train_file.split(".")[-1]
            assert extension in ["csv", "json"], "`train_file` should be a csv or a json file."
        if args.validation_file is not None:
            extension = args.validation_file.split(".")[-1]
            assert extension in ["csv", "json"], "`validation_file` should be a csv or a json file."
        if args.test_file is not None:
            extension = args.test_file.split(".")[-1]
            assert extension in ["csv", "json"], "`test_file` should be a csv or a json file."

    if args.output_dir is not None:
        os.makedirs(args.output_dir, exist_ok=True)
    
    ## 위에서 인자로 전달받은 값을 args 변수에 담아 반환
    return args

In [ ]:
# SQuAD 평가를 위한 전처리 함수
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [ ]:
logger = logging.getLogger(__name__)
def main():
    args = easydict_args()

    # Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
    ## 가속화 모드를 지원하며, 해당 모드로 병목이 발생하는 메서드 부분을 대체해서 사용가능.
    accelerator = Accelerator()
    # Make one log on every process with the configuration for debugging.
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
    )
    logger.info(accelerator.state)

    # Setup logging, we only want one process per machine to log things on the screen.
    # accelerator.is_local_main_process is only True for one process per machine.
    logger.setLevel(logging.INFO if accelerator.is_local_main_process else logging.ERROR)
    if accelerator.is_local_main_process:
        datasets.utils.logging.set_verbosity_warning()
        transformers.utils.logging.set_verbosity_info()
    else:
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()

    # If passed along, set the training seed now.
    if args.seed is not None:
        set_seed(args.seed)

    # Get the datasets: you can either provide your own CSV/JSON/TXT training and evaluation files (see below)
    # or just provide the name of one of the public datasets available on the hub at https://huggingface.co/datasets/
    # (the dataset will be downloaded automatically from the datasets Hub).
    #
    # For CSV/JSON files, this script will use the column called 'text' or the first column if no column called
    # 'text' is found. You can easily tweak this behavior (see below).
    #
    # In distributed training, the load_dataset function guarantee that only one local process can concurrently
    # download the dataset.
    
    ## Huggingface의 datasets에 존재하는 데이터셋 명을 사용하는 경우에는 저장된 형식에 맞추어서 로드 (위의 링크에서 리스트 참조)
    if args.dataset_name is not None:
        # Downloading and loading a dataset from the hub.
        raw_datasets = load_dataset(args.dataset_name, args.dataset_config_name)
    ## 사용자 정의의 데이터셋을 사용하는 경우에는 아래의 조건문 실행. 
    else:
        data_files = {}
        if args.train_file is not None:
            data_files["train"] = args.train_file
        if args.validation_file is not None:
            data_files["validation"] = args.validation_file
        if args.test_file is not None:
            data_files["test"] = args.test_file
        extension = args.train_file.split(".")[-1]
        raw_datasets = load_dataset(extension, data_files=data_files, field="data")
    # See more about loading any type of standard or custom dataset (from files, python dict, pandas DataFrame, etc) at
    # https://huggingface.co/docs/datasets/loading_datasets.html.

    # Load pretrained model and tokenizer
    #
    # In distributed training, the .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.

    ## 지정한 모델의 설정에 맞추어서 환경 구성 요소 로드
    ## ex) BERT --> BERTConfig...
    if args.config_name:
        config = AutoConfig.from_pretrained(args.config_name)
    elif args.model_name_or_path:
        config = AutoConfig.from_pretrained(args.model_name_or_path)
    else:
        config = CONFIG_MAPPING[args.model_type]()
        logger.warning("You are instantiating a new config instance from scratch.")
    
    ## 지정한 모델의 설정에 맞추어서 토크나이저 로드
    ## ex) BERT --> BERTTokenizer ... 30,000개의 vocab와 word piece tokenzier를 곁들인...
    if args.tokenizer_name:
        tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, use_fast=True)
    elif args.model_name_or_path:
        tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=True)
    else:
        raise ValueError(
            "You are instantiating a new tokenizer from scratch. This is not supported by this script."
            "You can do it from another script, save it, and load it from here, using --tokenizer_name."
        )

    ## 지정한 모델 로드하기.
    ## 여기서는 사전 훈련된 모델 상단에 질의응답 TASK를 위한 레이어를 추가하고 미세조정훈련을 하기위해 ForQuestionAnswering을 불러온다.
    if args.model_name_or_path:
        model = AutoModelForQuestionAnswering.from_pretrained(
            args.model_name_or_path,
            from_tf=bool(".ckpt" in args.model_name_or_path),
            config=config,
        )
    else:
        logger.info("Training new model from scratch")
        model = AutoModelForQuestionAnswering.from_config(config)

    # Preprocessing the datasets.
    # Preprocessing is slighlty different for training and evaluation.
    
    ## 전처리하기 
    column_names = raw_datasets["train"].column_names
    print(column_names)
    #print(column_names)
    question_column_name = "question" if "question" in column_names else column_names[0]
    context_column_name = "context" if "context" in column_names else column_names[1]
    answer_column_name = "answers" if "answers" in column_names else column_names[2]

    # Padding side determines if we do (question|context) or (context|question).
    pad_on_right = tokenizer.padding_side == "right"

    ## 인자로 전달한 최대 입력 시퀀스 길이가 모델의 최대 입력 임베딩 길이보다 길 경우 에러 반환
    if args.max_seq_length > tokenizer.model_max_length:
        logger.warning(
            f"The max_seq_length passed ({args.max_seq_length}) is larger than the maximum length for the"
            f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}."
        )

    max_seq_length = min(args.max_seq_length, tokenizer.model_max_length)

    ##################

    # Training preprocessing
    ## 훈련 데이터셋에 대한 전처리 작업
    ## 패드의 방향을 오른쪽으로 하는 것이 일반적이지만, 최근에는 다른 방향에도 추가할 수 있도록 기능 추가.
    
    def prepare_train_features(examples):
        # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
        # in one example possible giving several features when a context is long, each of those features having a
        # context that overlaps a bit the context of the previous feature.
        tokenized_examples = tokenizer(
            examples[question_column_name if pad_on_right else context_column_name],
            examples[context_column_name if pad_on_right else question_column_name],
            truncation="only_second" if pad_on_right else "only_first",
            max_length=max_seq_length,
            stride=args.doc_stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length" if args.pad_to_max_length else False,
        )
        # Since one example might give us several features if it has a long context, we need a map from a feature to
        # its corresponding example. This key gives us just that.
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
        # The offset mappings will give us a map from token to character position in the original context. This will
        # help us compute the start_positions and end_positions.
        
        ## span의 여부 체크
        offset_mapping = tokenized_examples.pop("offset_mapping")

        #print("offset_mapping", offset_mapping)

        # Let's label those examples!
        ## span 가능하도록 본문에서 정답이 존재하는 시작점, 끝점을 매핑하여 저장
        tokenized_examples["start_positions"] = []
        tokenized_examples["end_positions"] = []

        for i, offsets in enumerate(offset_mapping):
            # We will label impossible answers with the index of the CLS token.
            input_ids = tokenized_examples["input_ids"][i]
            cls_index = input_ids.index(tokenizer.cls_token_id)

            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)

            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            answers = examples[answer_column_name][sample_index]
            # If no answers are given, set the cls_index as answer.
            ## answer span이 없을 경우에는 [CLS]로 대체
            if len(answers["answer_start"]) == 0:
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Start/end character index of the answer in the text.
                start_char = answers["answer_start"][0]
                end_char = start_char + len(answers["text"][0])

                # Start token index of the current span in the text.
                token_start_index = 0
                while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                    token_start_index += 1

                # End token index of the current span in the text.
                token_end_index = len(input_ids) - 1
                while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                    token_end_index -= 1

                # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
                if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                    tokenized_examples["start_positions"].append(cls_index)
                    tokenized_examples["end_positions"].append(cls_index)
                else:
                    # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                    # Note: we could go after the last offset if the answer is the last word (edge case).
                    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                        token_start_index += 1
                    tokenized_examples["start_positions"].append(token_start_index - 1)
                    while offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    tokenized_examples["end_positions"].append(token_end_index + 1)

        return tokenized_examples
    
    ######################
    ## 사용자 정의 훈련 데이터를 입력 인자로 사용하는 경우, --do_train을 입력 인자로 반드시 사용해야함.
    if "train" not in raw_datasets:
        raise ValueError("--do_train requires a train dataset")
    train_dataset = raw_datasets["train"]
    ## 디버깅용
    if args.max_train_samples is not None:
        # We will select sample from whole data if agument is specified
        train_dataset = train_dataset.select(range(args.max_train_samples))
    
    ## 로드한 사용자 정의 훈련 데이터셋을 입력 데이터로 인코딩
    # Create train feature from dataset
    train_dataset = train_dataset.map(
        prepare_train_features,
        batched=True,
        num_proc=args.preprocessing_num_workers,
        remove_columns=column_names,
        load_from_cache_file=not args.overwrite_cache,
    )
    if args.max_train_samples is not None:
        # Number of samples might increase during Feature Creation, We select only specified max samples
        train_dataset = train_dataset.select(range(args.max_train_samples))

    ######################
    ## 위의 훈련 데이터와 동일
    # Validation preprocessing
    def prepare_validation_features(examples):
        # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
        # in one example possible giving several features when a context is long, each of those features having a
        # context that overlaps a bit the context of the previous feature.
        tokenized_examples = tokenizer(
            examples[question_column_name if pad_on_right else context_column_name],
            examples[context_column_name if pad_on_right else question_column_name],
            truncation="only_second" if pad_on_right else "only_first",
            max_length=max_seq_length,
            stride=args.doc_stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length" if args.pad_to_max_length else False,
        )

        # Since one example might give us several features if it has a long context, we need a map from a feature to
        # its corresponding example. This key gives us just that.
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

        # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
        # corresponding example_id and we will store the offset mappings.
        tokenized_examples["example_id"] = []

        for i in range(len(tokenized_examples["input_ids"])):
            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)
            context_index = 1 if pad_on_right else 0

            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            tokenized_examples["example_id"].append(examples["id"][sample_index])

            # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
            # position is part of the context or not.
            tokenized_examples["offset_mapping"][i] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(tokenized_examples["offset_mapping"][i])
            ]

        return tokenized_examples

    #################
    ## 사용자 정의 검증 데이터를 입력 인자로 사용하는 경우, --do_eval을 입력 인자로 반드시 사용해야함.
    if "validation" not in raw_datasets:
        raise ValueError("--do_eval requires a validation dataset")
    eval_examples = raw_datasets["validation"]
    if args.max_eval_samples is not None:
        # We will select sample from whole data
        eval_examples = eval_examples.select(range(args.max_eval_samples))
    # Validation Feature Creation
    eval_dataset = eval_examples.map(
        prepare_validation_features,
        batched=True,
        num_proc=args.preprocessing_num_workers,
        remove_columns=column_names,
        load_from_cache_file=not args.overwrite_cache,
    )

    if args.max_eval_samples is not None:
        # During Feature creation dataset samples might increase, we will select required samples again
        eval_dataset = eval_dataset.select(range(args.max_eval_samples))

    #################
    ## 실제 정답을 반환하고 싶을 경우
    ## --do_predict을 입력 인자로 전달하는 경우, 사용자 정의 평가 데이터를 입력 인자로 전달해야함. 
    if args.do_predict:
        if "test" not in raw_datasets:
            raise ValueError("--do_predict requires a test dataset")
        predict_examples = raw_datasets["test"]
        if args.max_predict_samples is not None:
            # We will select sample from whole data
            predict_examples = predict_examples.select(range(args.max_predict_samples))
        # Predict Feature Creation
        predict_dataset = predict_examples.map(
            prepare_validation_features,
            batched=True,
            num_proc=args.preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=not args.overwrite_cache,
        )
        if args.max_predict_samples is not None:
            # During Feature creation dataset samples might increase, we will select required samples again
            predict_dataset = predict_dataset.select(range(args.max_predict_samples))

    # Log a few random samples from the training set:
    for index in random.sample(range(len(train_dataset)), 3):
        logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

    #################
    ## 입력 인코딩을 완료한 데이터셋에 대해서 batch_size 만큼 로드하기 (pytorch 프레임워크의 장점)
    # DataLoaders creation:
    if args.pad_to_max_length:
        # If padding was already done ot max length, we use the default data collator that will just convert everything
        # to tensors.
        data_collator = default_data_collator
    else:
        # Otherwise, `DataCollatorWithPadding` will apply dynamic padding for us (by padding to the maximum length of
        # the samples passed). When using mixed precision, we add `pad_to_multiple_of=8` to pad all tensors to multiple
        # of 8s, which will enable the use of Tensor Cores on NVIDIA hardware with compute capability >= 7.5 (Volta).
        data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=(8 if accelerator.use_fp16 else None))

    ## 훈련 데이터셋에 대한 로더 (지정한 배치 사이즈 만큼만 로드해서 훈련하기)
    train_dataloader = DataLoader(
        train_dataset, shuffle=True, collate_fn=data_collator, batch_size=args.per_device_train_batch_size
    )

    ## 검증 데이터셋에 대한 로더
    eval_dataset_for_model = eval_dataset.remove_columns(["example_id", "offset_mapping"])
    eval_dataloader = DataLoader(
        eval_dataset_for_model, collate_fn=data_collator, batch_size=args.per_device_eval_batch_size
    )

    ## 실제 평가를 위해서 정답의 근거를 삭제한 형태로 inference
    if args.do_predict:
        predict_dataset_for_model = predict_dataset.remove_columns(["example_id", "offset_mapping"])
        predict_dataloader = DataLoader(
            predict_dataset_for_model, collate_fn=data_collator, batch_size=args.per_device_eval_batch_size
        )

    ## huggingface에서 제공하는 간편하게 평가 결과를 반환하도록 하는 함수 
    ## 깃헙에서 다운로드 받은 utils_qa.py를 참조
    # Post-processing:
    def post_processing_function(examples, features, predictions, stage="eval"):
        # Post-processing: we match the start logits and end logits to answers in the original context.
        predictions = postprocess_qa_predictions(
            examples=examples,
            features=features,
            predictions=predictions,
            version_2_with_negative=args.version_2_with_negative,
            n_best_size=args.n_best_size,
            max_answer_length=args.max_answer_length,
            null_score_diff_threshold=args.null_score_diff_threshold,
            output_dir=args.output_dir,
            prefix=stage,
        )
        # Format the result to the format the metric expects.
        if args.version_2_with_negative:
            formatted_predictions = [
                {"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in predictions.items()
            ]
        else:
            formatted_predictions = [{"id": k, "prediction_text": v} for k, v in predictions.items()]
            #formatted_predictions = [{"id": normalize_answer(k), "prediction_text": normalize_answer(v)} for k, v in predictions.items()]
            df_pred = pd.DataFrame(formatted_predictions)
            df_pred.to_csv(args.test_save_path , index=False, encoding='utf-8')

        references = [{"id": ex["id"], "answers": ex[answer_column_name]} for ex in examples]
        return EvalPrediction(predictions=formatted_predictions, label_ids=references)

    ## F1 score, Exact match
    metric = load_metric("squad_v2" if args.version_2_with_negative else "squad")

    #################
    # Create and fill numpy array of size len_of_validation_data * max_length_of_output_tensor
    def create_and_fill_np_array(start_or_end_logits, dataset, max_len):
        """
        Create and fill numpy array of size len_of_validation_data * max_length_of_output_tensor
        Args:
            start_or_end_logits(:obj:`tensor`):
                This is the output predictions of the model. We can only enter either start or end logits.
            eval_dataset: Evaluation dataset
            max_len(:obj:`int`):
                The maximum length of the output tensor. ( See the model.eval() part for more details )
        """

        step = 0
        # create a numpy array and fill it with -100.
        ## pad에 해당하는 logit 값을 -100으로 지정
        ## 효율적인 배치 계산을 위한 방법으로 지정한 max sequence length에 메모리 효율을 올리는 방법
        logits_concat = np.full((len(dataset), max_len), -100, dtype=np.float64)
        # Now since we have create an array now we will populate it with the outputs gathered using accelerator.gather
        for i, output_logit in enumerate(start_or_end_logits):  # populate columns
            # We have to fill it such that we have to take the whole tensor and replace it on the newly created array
            # And after every iteration we have to change the step

            batch_size = output_logit.shape[0]
            cols = output_logit.shape[1]

            if step + batch_size < len(dataset):
                logits_concat[step : step + batch_size, :cols] = output_logit
            else:
                logits_concat[step:, :cols] = output_logit[: len(dataset) - step]

            step += batch_size

        return logits_concat

    # Optimizer
    # Split weights in two groups, one with weight decay and the other not.
    ## 최적화 과정에서 L2 Loss를 통해서 지나친 과적합을 방지하기 위한 부분. 
    ## L2 Regularization, 기존 손실 함수 값에 모든 학습 가능한 파라미터를 제곱하여 합하고 
    ## 기정한 람다 값만큼 영향도를 설정하는데 이때 람다는 정규화의 세기를 결정하는 하이퍼 파라미터로 
    ## 여기서는 weight_decay에 해당
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    ## 아담 최적화 함수 사용
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate)

    # Prepare everything with our `accelerator`.
    model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
        model, optimizer, train_dataloader, eval_dataloader
    )

    # Note -> the training dataloader needs to be prepared before we grab his length below (cause its length will be
    # shorter in multiprocess)

    # Scheduler and math around the number of training steps.
    ## 가상의 미니배치사용. 최종 업데이트는 각 가상 미니 배치의 gradient 값을 모두 고려한 global gradient
    num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)
    if args.max_train_steps is None:
        args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
    else:
        args.num_train_epochs = math.ceil(args.max_train_steps / num_update_steps_per_epoch)

    ## scheduler를 통해서 learning rate의 속도 조절.
    ## 처음에는 크게, 점점 약해지도록하며 local minimum으로 추정되는 구간에서는 크게 지정한 횟수 만큼 적용 가능.
    lr_scheduler = get_scheduler(
        name=args.lr_scheduler_type,
        optimizer=optimizer,
        num_warmup_steps=args.num_warmup_steps,
        num_training_steps=args.max_train_steps,
    )

    # Train!
    total_batch_size = args.per_device_train_batch_size * accelerator.num_processes * args.gradient_accumulation_steps

    logger.info("***** Running training *****")
    logger.info(f"  Num examples = {len(train_dataset)}")
    logger.info(f"  Num Epochs = {args.num_train_epochs}")
    logger.info(f"  Instantaneous batch size per device = {args.per_device_train_batch_size}")
    logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
    logger.info(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
    logger.info(f"  Total optimization steps = {args.max_train_steps}")

    # Only show the progress bar once on each machine.
    progress_bar = tqdm(range(args.max_train_steps), disable=not accelerator.is_local_main_process)
    completed_steps = 0

    for epoch in range(args.num_train_epochs):
        model.train()
        for step, batch in enumerate(train_dataloader):
            outputs = model(**batch)
            loss = outputs.loss
            loss = loss / args.gradient_accumulation_steps
            accelerator.backward(loss)
            if step % args.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
                progress_bar.update(1)
                completed_steps += 1

            if completed_steps >= args.max_train_steps:
                break

    # Validation
    all_start_logits = []
    all_end_logits = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)
            start_logits = outputs.start_logits # 모든 토큰 중에서 가장 probability가 높은 시작점
            end_logits = outputs.end_logits# 모든 토큰 중에서 가장 probability가 높은 끝점
            
            if not args.pad_to_max_length:  # necessary to pad predictions and labels for being gathered
                start_logits = accelerator.pad_across_processes(start_logits, dim=1, pad_index=-100)
                end_logits = accelerator.pad_across_processes(end_logits, dim=1, pad_index=-100)

            all_start_logits.append(accelerator.gather(start_logits).cpu().numpy())
            all_end_logits.append(accelerator.gather(end_logits).cpu().numpy())

    max_len = max([x.shape[1] for x in all_start_logits])  # Get the max_length of the tensor

    # concatenate the numpy array
    start_logits_concat = create_and_fill_np_array(all_start_logits, eval_dataset, max_len)
    end_logits_concat = create_and_fill_np_array(all_end_logits, eval_dataset, max_len)

    # delete the list of numpy arrays
    del all_start_logits
    del all_end_logits

    outputs_numpy = (start_logits_concat, end_logits_concat)
    prediction = post_processing_function(eval_examples, eval_dataset, outputs_numpy)
    eval_metric = metric.compute(predictions=prediction.predictions, references=prediction.label_ids)
    logger.info(f"Evaluation metrics: {eval_metric}")

    # Prediction
    '''
    if args.do_predict:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        all_start_logits = []
        all_end_logits = []
        for step, batch in enumerate(predict_dataloader):
            batch = batch.to(device)
            with torch.no_grad():
                outputs = model(**batch)
                start_logits = outputs.start_logits
                end_logits = outputs.end_logits

                if not args.pad_to_max_length:  # necessary to pad predictions and labels for being gathered
                    start_logits = accelerator.pad_across_processes(start_logits, dim=1, pad_index=-100)
                    end_logits = accelerator.pad_across_processes(start_logits, dim=1, pad_index=-100)

                all_start_logits.append(accelerator.gather(start_logits).cpu().numpy())
                all_end_logits.append(accelerator.gather(end_logits).cpu().numpy())

        max_len = max([x.shape[1] for x in all_start_logits])  # Get the max_length of the tensor
        # concatenate the numpy array
        start_logits_concat = create_and_fill_np_array(all_start_logits, predict_dataset, max_len)
        end_logits_concat = create_and_fill_np_array(all_end_logits, predict_dataset, max_len)

        # delete the list of numpy arrays
        del all_start_logits
        del all_end_logits

        outputs_numpy = (start_logits_concat, end_logits_concat)
        prediction = post_processing_function(predict_examples, predict_dataset, outputs_numpy)
        predict_metric = metric.compute(predictions=prediction.predictions, references=prediction.label_ids)
        logger.info(f"Predict metrics: {predict_metric}")
    '''
    #if args.output_dir is not None:
        #accelerator.wait_for_everyone()
        #unwrapped_model = accelerator.unwrap_model(model)
        #unwrapped_model.save_pretrained(args.output_dir, save_function=accelerator.save)



In [ ]:
if __name__ == '__main__':
    main()

In [ ]:
# Tip!
'''
with open('final_test_no_context.json', 'r', encoding='utf-8') as f:
    test_dataset = json.load(f)
    for line, retrieved_line in zip(test_dataset['data'], retrieved_data):
        line['context'] = retrieved_line # what you retrieved from document-based database

with open('final_test_with_retrieved_text.json', 'w', encoding='utf-8') as f:
    json.dump(test_dataset, f)
    
'''

## 👍 실습 종료!! 정말 고생 많으셨습니다.